# Bitap Algorithm (Levenshtein Distance)

The following notebook is used for evaluating approximate matching at the sentence level, where the text has 

In [5]:
from fuzzywuzzy import fuzz
import random

## Homoglyph substitution

In [1]:
unicode_dict = {
    'a': ['а', 'ạ', 'ą', 'ä', 'à', 'á', 'ą'],
    'c': ['с', 'ƈ', 'ċ'],
    'd': ['ԁ', 'ɗ'],
    'e': ['е', 'ẹ', 'ė', 'é', 'è'],
    'g': ['ġ'],
    'h': ['һ'],
    'i': ['і', 'í', 'ï'],
    'j': ['ј', 'ʝ'],
    'k': ['κ'],
    'l': ['ӏ', 'ḷ'],
    'n': ['ո'],
    'o': ['о', 'ο', 'օ', 'ȯ', 'ọ', 'ỏ', 'ơ', 'ó', 'ò', 'ö'],
    'p': ['р'],
    'q': ['զ'],
    's': ['ʂ'],
    'u': ['υ', 'ս', 'ü', 'ú', 'ù'],
    'v': ['ν', 'ѵ'],
    'x': ['х', 'ҳ'],
    'y': ['у', 'ý'],
    'z': ['ʐ', 'ż']
}

In [20]:
def unicode_text_converter(text, unicode_dict, max_changes=20):
    """
    Convert characters in the input text to Unicode look-alikes.
    
    Args:
    text (str): The input text to be modified.
    unicode_dict (dict): A dictionary mapping characters to lists of Unicode look-alikes.
    max_changes (int): The maximum number of character changes to make. Defaults to 20.
    
    Returns:
    tuple: A tuple containing the modified text (str) and the number of changes made (int).
    """
    char_list = list(text)
    changes_made = 0
    
    for i, char in enumerate(char_list):
        if changes_made >= max_changes:
            break
        
        if char.lower() in unicode_dict and unicode_dict[char.lower()]:
            if random.random() < 0.5:
                unicode_char = random.choice(unicode_dict[char.lower()])
                
                if char.isupper():
                    char_list[i] = unicode_char.upper()
                else:
                    char_list[i] = unicode_char
                
                changes_made += 1
    
    modified_text = ''.join(char_list)
    
    return modified_text, changes_made



## Invisible character insertion
The following method intentionally modifies the text by replacing some spaces with special characters, joining words together. By choosing a list of hardly used special characters to use as replacements, this would make it easier for the plagiariser to identify and modify the font color to white - giving the appearance of separate words while tricking NLP methods that are attempting to match text for detection. 


In [22]:

special_chars = ['æ', 'œ', 'ß', 'ø', 'þ', 'ð', 'ŋ', 'ħ', 'ĸ', 'ł', 'đ', 'ſ', 'ŧ', 'ŋ']


In [23]:
def specchar_space_converter(text, max_changes=20):
    """
    Replace spaces between words with special characters in the input text.

    Args:
    text (str): The input text to be modified.
    max_changes (int): The maximum number of space replacements to make. Defaults to 20.

    Returns:
    tuple: A tuple containing the modified text (str) and the number of changes made (int).
    """
    words = text.split()
    changes_made = 0
    modified_words = []
    
    for i in range(len(words) - 1):
        modified_words.append(words[i])
        
        if changes_made < max_changes and random.random() < 0.5:
            special_char = random.choice(special_chars)
            modified_words.append(special_char)
            changes_made += 1
        else:
            modified_words.append(' ')
    
    modified_words.append(words[-1])
    modified_text = ''.join(modified_words)
    
    return modified_text, changes_made

## Evaluation

### Test Data

In [ ]:
text_test = """Spatial inference, or estimation, of a quantity Z : : R n → → R {{\\displaystyle Z\\colon \\mathbb {{R}} ^{{n}}\\to \\mathbb {{R}} }} , at an unobserved location x 0 {{\\displaystyle x_{{0}}}} , is calculated from a linear combination of the observed values z i = Z ( x i ) {{\\displaystyle z_{{i}}=Z(x_{{i}})}} and weights w i ( x 0 ) , i = 1 , … … , N {{\\displaystyle w_{{i}}(x_{{0}}),\\;i=1,\\ldots ,N}} : The weights w i {{\\displaystyle w_{{i}}}} are intended to summarize two extremely important procedures in a spatial inference process: reflect the structural "proximity" of samples to the estimation location x 0 {{\\displaystyle x_{{0}}}} ; at the same time, they should have a desegregation effect, in order to avoid bias caused by eventual sample clusters. When calculating the weights w i {{\\displaystyle w_{{i}}}} , there are two objectives in the geostatistical formalism: unbias and minimal variance of estimation. If the cloud of real values Z ( x 0 ) {{\\displaystyle Z(x_{{0}})}} is plotted against the estimated values Z ^ ^ ( x 0 ) {{\\displaystyle {{\\hat {{Z}}}}(x_{{0}})}} , the criterion for global unbias, intrinsic stationarity or [wide sense stationarity](https://en.wikipedia.org/wiki/Stationary_process) of the field, implies that the mean of the estimations must be equal to mean of the real values. The second criterion says that the mean of the squared deviations ( Z ^ ^ ( x ) − − Z ( x ) ) {{\\displaystyle {{\\big (}}{{\\hat {{Z}}}}(x)-Z(x){{\\big }}}} must be minimal, which means that when the cloud of estimated values versus the cloud real values is more disperse, the estimator is more imprecise."""

### Evaluation Metrics

In [24]:
def calculate_matching_score(original_text, modified_text):
    """
    Calculate the similarity score between original and modified texts.

    Args:
    original_text (str): The original unmodified text.
    modified_text (str): The modified version of the text.

    Returns:
    float: The average similarity score between corresponding words,
           normalized to a 0-1 scale.
    """
    original_words = original_text.split()
    modified_words = modified_text.split()
    
    total_score = 0
    for orig_word, mod_word in zip(original_words, modified_words):
        total_score += fuzz.ratio(orig_word.lower(), mod_word.lower())
    
    average_score = total_score / len(original_words)
    return average_score / 100

In [18]:
def classify_score(score):
    if score >= 0.95:
        return "Nearly Identical", "The texts are practically the same with minimal changes."
    elif score >= 0.90:
        return "Very Similar", "The texts are very similar with only minor differences."
    elif score >= 0.80:
        return "Similar", "The texts are similar but have noticeable differences."
    elif score >= 0.70:
        return "Moderately Different", "The texts have significant differences but maintain overall structure."
    elif score >= 0.50:
        return "Substantially Different", "The texts have major differences but some similarities remain."
    else:
        return "Very Different", "The texts are drastically different or unrelated."



#### Unicode Insertion

In [ ]:
def unicode_text_converter_with_score(text, unicode_dict, max_changes=50):
    """
    Convert text using Unicode look-alikes and calculate similarity score.

    Args:
    text (str): The input text to be modified.
    unicode_dict (dict): A dictionary mapping characters to Unicode look-alikes.
    max_changes (int): Maximum number of character changes. Defaults to 50.

    Returns:
    tuple: Contains modified text (str), number of changes (int),
           matching score (float), classification (str), and description (str).
    """
    modified_text, num_changes = unicode_text_converter(text, unicode_dict, max_changes)
    matching_score = calculate_matching_score(text, modified_text)
    classification, description = classify_score(matching_score)
    return modified_text, num_changes, matching_score, classification, description

In [26]:
text = text_test
modified_text, num_changes, matching_score, classification, description = unicode_text_converter_with_score(text, unicode_dict)
#print(f"Original text: {text}")
print(f"Modified text: {modified_text}")
print(f"Number of changes made: {num_changes}")
print(f"Matching score: {matching_score:.2f}")
print(f"Classification: {classification}")
print(f"Description: {description}")

Modified text: Ʂрatiаl іոfėrẹnce, ȯr estímàtion, оf ạ qùạntïty Z : : R ո → → R {{\diʂрlạystyle ʐ\coӏòn \mąthbb {{R}} ^{{ո}}\to \mạtһbb {{R}} }} , ąt an unοbserѵeɗ locatiơn x 0 {{\dіʂрḷaýʂtyḷе x_{{0}}}} , is ƈaḷculаtẹԁ frօm a lineаr combinatïօn of thė ơbserved values z i = Z ( x i ) {{\displaystyle z_{{i}}=Z(x_{{i}})}} and weights w i ( x 0 ) , i = 1 , … … , N {{\displaystyle w_{{i}}(x_{{0}}),\;i=1,\ldots ,N}} : The weights w i {{\displaystyle w_{{i}}}} are intended to summarize two extremely important procedures in a spatial inference process: reflect the structural "proximity" of samples to the estimation location x 0 {{\displaystyle x_{{0}}}} ; at the same time, they should have a desegregation effect, in order to avoid bias caused by eventual sample clusters. When calculating the weights w i {{\displaystyle w_{{i}}}} , there are two objectives in the geostatistical formalism: unbias and minimal variance of estimation. If the cloud of real values Z ( x 0 ) {{\displaystyle Z(x_{{0}})}

#### Hidden special characters

In [ ]:
def specchar_space_converter_with_score(text, max_changes=20):
    """
    Replace spaces with special characters and calculate similarity score.

    Args:
    text (str): The input text to be modified.
    max_changes (int): Maximum number of space replacements. Defaults to 20.

    Returns:
    tuple: Contains modified text (str), number of changes (int),
           matching score (float), classification (str), and description (str).
    """
    modified_text, num_changes = specchar_space_converter(text, max_changes)
    matching_score = calculate_matching_score(text, modified_text)
    classification, description = classify_score(matching_score)
    return modified_text, num_changes, matching_score, classification, description

In [27]:
text = text_test
modified_text, num_changes, matching_score, classification, description = specchar_space_converter_with_score(text)
#print(f"Original text: {text}")
print(f"Modified text: {modified_text}")
print(f"Number of changes made: {num_changes}")
print(f"Matching score: {matching_score:.2f}")
print(f"Classification: {classification}")
print(f"Description: {description}")

Modified text: Spatial inference, orŧestimation,ŋof aſquantity Z : :ſR nĸ→æ→ R {{\displaystyle Z\colon \mathbb {{R}} ^{{n}}\to \mathbb {{R}}ŋ}}đ, at anøunobservedælocationðxĸ0ħ{{\displaystyleŋx_{{0}}}} ,æis calculatedĸfrom ađlinear combination of the observed values z i = Z (ŧxſi ) {{\displaystyleøz_{{i}}=Z(x_{{i}})}} and weights w i ( x 0 ) , i = 1 , … … , N {{\displaystyle w_{{i}}(x_{{0}}),\;i=1,\ldots ,N}} : The weights w i {{\displaystyle w_{{i}}}} are intended to summarize two extremely important procedures in a spatial inference process: reflect the structural "proximity" of samples to the estimation location x 0 {{\displaystyle x_{{0}}}} ; at the same time, they should have a desegregation effect, in order to avoid bias caused by eventual sample clusters. When calculating the weights w i {{\displaystyle w_{{i}}}} , there are two objectives in the geostatistical formalism: unbias and minimal variance of estimation. If the cloud of real values Z ( x 0 ) {{\displaystyle Z(x_{{0}})}

#### Summarisation, paraphrasing, and rewriting 

In [29]:
paraphrase_text = "'Spatial inference involves estimating a quantity Z at an unobserved location x0 by using a linear combination of observed values Z(xi) and weights wi(x0). These weights play a crucial role in summarizing two key aspects in spatial inference: reflecting sample proximity to the estimation location x0 and preventing bias due to sample clustering. In geostatistical formalism, the objectives for calculating weights are to ensure unbiased estimates and minimize estimation variance. Meeting the criteria of global unbias and field stationarity involves ensuring the mean of estimations matches the mean of real values, and minimizing the mean of squared deviations between estimated and real values indicates precision.'"

In [ ]:
def paraphrase_with_score(text):
    """
    Paraphrase the input text and calculate its similarity score.

    Args:
    text (str): The input text to be paraphrased.

    Returns:
    tuple: Contains paraphrased text (str), matching score (float),
           classification (str), and description (str).
    """
    modified_text = paraphrase_text(text)
    matching_score = calculate_matching_score(text, modified_text)
    classification, description = classify_score(matching_score)
    return modified_text, matching_score, classification, description

In [30]:
text = text_test
modified_text, matching_score, classification, description = paraphrase_with_score(text)
#print(f"Original text: {text}")
print(f"Modified text: {modified_text}")
print(f"Matching score: {matching_score:.2f}")
print(f"Classification: {classification}")
print(f"Description: {description}")

Modified text: 'Spatial inference involves estimating a quantity Z at an unobserved location x0 by using a linear combination of observed values Z(xi) and weights wi(x0). These weights play a crucial role in summarizing two key aspects in spatial inference: reflecting sample proximity to the estimation location x0 and preventing bias due to sample clustering. In geostatistical formalism, the objectives for calculating weights are to ensure unbiased estimates and minimize estimation variance. Meeting the criteria of global unbias and field stationarity involves ensuring the mean of estimations matches the mean of real values, and minimizing the mean of squared deviations between estimated and real values indicates precision.'
Matching score: 0.05
Classification: Very Different
Description: The texts are drastically different or unrelated.


## Conclusion
Bitap and/or Approximate Matching scored as follows:

1. Unicode Insertion: Matching scores >0.95 
2. Hidden special characters: Matching scores 0.05-0.10
3. Summarisation, paraphrasing, and rewriting: Matching scores <0.05


Testing revealed that Bitap is very strong for minor changes such as unicode due to the limited edit distance required in the characters. It was classified as "Nearly Identical" i.e. the texts are practically the same with minimal changes.
In contrast, Approximate Matching was poor when hidden special characters were added as it caused the word lengths and overall structure to change significantly. The results were even worse when any summarisation, rewriting, or paraphrasing was done.
